In [0]:
import streamlit as st
import yaml, os, subprocess

# CONFIGS_DIR on DBFS, e.g. /dbfs/configs
CONFIGS_DIR = "/dbfs/configs"

st.title("Autoloader Pipeline Configurator")

# 1. Gather inputs
name = st.text_input("Pipeline name", value="example_pipeline")
source = st.text_input("Source path", "/mnt/raw-data/streaming-input")
target = st.text_input("Target base path", "/mnt/processed-data")
fmt = st.selectbox("File format", ["csv","json","parquet"])
have_hdr = st.checkbox("Has header row?", True)
layer = st.selectbox("Medallion layer", ["bronze","silver","gold"])
area = st.text_input("Subject area", "investments")
mode = st.radio("Mode", ["create","append","overwrite"])

# 2. Validation
if mode in ("append","overwrite") and layer=="gold":
    st.error("Gold layer must be 'create' mode only.")

if st.button("Generate Config"):
    cfg = {
      "pipeline_name": name,
      "source_path": source,
      "target_base": target,
      "format": fmt,
      "schema": {"hasHeader": have_hdr},
      "medallion": layer,
      "subject_area": area,
      "mode": mode
    }
    path = os.path.join(CONFIGS_DIR, f"{name}.yaml")
    with open(path, "w") as f:
        yaml.safe_dump(cfg, f)
    st.success(f"Config written to `{path}`")
    st.code(yaml.safe_dump(cfg), language="yaml")

    # Optional: commit to Git
    # subprocess.run(["git","add", path])
    # subprocess.run(["git","commit","-m", f"Add config {name}"])
    # subprocess.run(["git","push"])
